# Residual error automatic checker
This notebook recursively find all residual errors files (default: ResMS.hdr|nii) and computes various descriptive stats measures and raise an alert if a measure is reaching above a preset threshold.

Note that all stats and alerts are on non-zeros values only, in other words you should not count the quantity of 0 values in your calculations (also don't be afraid if you don't see any 0 value in histograms, that's normal, the count of 0 values are provided in descriptive stats for your information).

By Stephen Larroque from Coma Science Group, University of Liège, created on 2018-01-27.

Version v1.2

In [ ]:
%load_ext autoreload
%autoreload 2
# BEWARE: autoreload works on functions and on general code, but NOT on new class methods:
# if you add or change the name of a method, you have to reload the kernel!
# also it will fail if you use super() calls in the classes you change
# ALSO AUTORELOAD SHOULD BE THE FIRST LINE EVER EXECUTED IN YOUR IPYTHON NOTEBOOK!!!

# Profilers:
# http://pynash.org/2013/03/06/timing-and-profiling/
# http://mortada.net/easily-profile-python-code-in-jupyter.html
# use %lprun -m module func(*args, **kwargs)
try:
    %load_ext line_profiler
    %load_ext memory_profiler
except ImportError as exc:
    pass

In [ ]:
# Generate figure inside IPython Notebook (must be called before any import of matplotlib, direct or indirect!)
%matplotlib inline

In [ ]:
import pprint
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import os
import re
import scipy.stats
from nilearn import image
from nilearn import plotting
#try:
#    from niwidgets import NiftiWidget
#    niwidgets_available = True
#except Exception:
#    niwidgets_available = False
#    pass

try:
    from scandir import walk # use the faster scandir module if available (Python >= 3.5), see https://github.com/benhoyt/scandir
except ImportError as exc:
    from os import walk # else, default to os.walk()

In [ ]:
# PARAMETERS - EDIT ME
rootpath = r'Z:\some_folder'  # path from where to start recursively looking for residual error files
RE_resms = 'ResMS\.(img|nii)'  # regular expression to match residual error filenames
alert_thresholds = {'max': 0.2, 'mean': 0.05, 'pct99': 0.1}  # alert thresholds, can use any descriptive stat calculated below. Must be formatted as a dictionary with 'parameter': threshold-value
verbose = False  # in verbose mode, descriptive stats and histograms of all files will be displayed, whether it raises an alert or not

In [ ]:
def recwalk(inputpath, sorting=True, folders=False, topdown=True, filetype=None, regex=None):
    '''Recursively walk through a folder. This provides a mean to flatten out the files restitution (necessary to show a progress bar). This is a generator.'''
    if filetype and isinstance(filetype, list):
        filetype = tuple(filetype)  # str.endswith() only accepts a tuple, not a list
    if regex:
        RE_precomp = re.compile(regex, re.I)
    # If it's only a single file, return this single file
    if os.path.isfile(inputpath):
        abs_path = fullpath(inputpath)
        yield os.path.dirname(abs_path), os.path.basename(abs_path)
    # Else if it's a folder, walk recursively and return every files
    else:
        for dirpath, dirs, files in walk(inputpath, topdown=topdown):
            if sorting:
                files.sort()
                dirs.sort()  # sort directories in-place for ordered recursive walking
            # return each file
            for filename in files:
                if (not filetype or filename.endswith(filetype)) and (not regex or RE_precomp.search(filename)):
                    yield (dirpath, filename)  # return directory (full path) and filename
            # return each directory
            if folders:
                for folder in dirs:
                    yield (dirpath, folder)

In [ ]:
voxel_threshold = 0.0001 # minimum threshold to consider as a voxel and not just background noise (because background voxels can be 0.000001 for example), can be float or str ('1%' to give a percentage). TODO: autodetect minimum value (can be -4, 0.02, etc) as the background and use it as the threshold value.
for dirpath, filename in recwalk(rootpath, regex=RE_resms):
    # Get fullpath of image
    fpath = os.path.join(dirpath, filename)
    # Load image data
    im = image.load_img(fpath)
    # Remove background noise voxels    
    im = image.threshold_img(im, voxel_threshold)
    imdata = im.get_data()
    # Calculate descriptive stats
    imdata_nz = imdata[imdata != 0]  # only on non-zeros values
    desc_stats = {'mean': imdata_nz.mean(),
                  'median': np.median(imdata_nz),
                  'min': imdata_nz.min(),
                  'max': imdata_nz.max(),
                  'range': np.ptp(imdata_nz),
                  'pct25': np.percentile(imdata_nz,25,interpolation='lower'),
                  'pct50': np.percentile(imdata_nz,50,interpolation='lower'),
                  'pct75': np.percentile(imdata_nz,75,interpolation='lower'),
                  'pct90': np.percentile(imdata_nz,90,interpolation='lower'),
                  'pct99': np.percentile(imdata_nz,99,interpolation='lower'),
                  'iqr': scipy.stats.iqr(imdata_nz, rng=(25, 75), interpolation='lower'),
                  'var': np.var(imdata_nz),
                  'stdev': np.std(imdata_nz),
                  'skew': scipy.stats.skew(imdata_nz),
                  'kurtosis': scipy.stats.kurtosis(imdata_nz, fisher=True),
                  'count_nonzero': np.size(imdata_nz),
                  'count': np.size(imdata),
                  'count_zero': np.size(imdata[imdata == 0]),
                  }
    # Check if we reached aboved alert thresholds on any descriptive stats feature
    alert = False
    if alert_thresholds is not None:
        for feature, threshold in alert_thresholds.items():
            if desc_stats[feature] > threshold:
                alert = True
                break
    # If the alert is raised or we are in verbose mode, display infos about this file
    if alert or verbose:
        # Display which file raised the alert (useful for user to find the incriminated file)
        if alert:
            print('Alert! Residual errors are above thresholds in file: %s' % fpath)
        else:
            print('Displaying stats for residual error file: %s' % fpath)
        # Print descriptive stats
        pprint.pprint(desc_stats)
        # Show histogram
        frq, edges = np.histogram(imdata_nz, bins=100)
        fig, ax = plt.subplots()
        ax.bar(edges[:-1], frq, width=np.diff(edges), ec="k", align="edge")
        plt.xlabel('Residual error value voxel-wise')
        plt.ylabel('Occurrences')
        plt.show()
        # Show spatial localization on brain of the residual errors
        # If niwidgets is available, show an interactive visualization, else just show nilearn static visualization of the max value
        #if niwidgets_available:
            #my_widget = NiftiWidget(fpath)
            #my_widget.nifti_plotter()
        #else:
        fig2 = plotting.plot_glass_brain(fpath, colorbar=True)
        plt.show()